In [3]:
import numpy as np
import matplotlib.pyplot as plt
import pandas as pd

In [4]:
source_path = "/home/vincent/Documenten/BachelorsProject/qualitative/"
csv_file = "/home/vincent/Documenten/BachelorsProject/GitHub_Repo/data_annotations/qualitative.csv"
target_dir = "/home/vincent/Documenten/BachelorsProject/GitHub_Repo/results/qualitative/"

In [5]:
# Translation table, converting labels from Dutch to English
trans_tab = {
    "prent": "engraving",
    "tekening": "drawing",
    "schotel": "dish",
    "bord (vaatwerk)": "plate",
    "foto": "picture",
    "schilderij": "painting",
    "tekstblad": "writing",
    "pot": "pot",
    "vaas": "vase",
    "fles": "bottle",
    "beeldhouwwerk": "sculpture",
    "demonstratiemodel": "demo model",
    "miniatuur": "miniature"
}

In [6]:
def stringFromFile(file):
    with open(file, "r") as f:
        ret = f.read()
    return ret

In [7]:
def plotSalience(gray, salience, prediction, x, y):
    assert prediction in trans_tab, f"{prediction} is not in the table yet!"

    salience = np.maximum(salience, np.quantile(salience, 0.3))
    salience = np.minimum(salience, np.quantile(salience, 0.95))

    plt.imshow(gray, cmap="gray", extent=[x,x+1,y,y+1]) # left, right, bottom, top
    plt.imshow(salience, extent=[x,x+1,y,y+1], alpha=0.8)
    plt.text(x+0.05, y+0.08, trans_tab[prediction], bbox={'facecolor': 'white', 'alpha':0.7})

In [8]:
true_labels = pd.read_csv(csv_file)["material"].to_list()
true_labels = [trans_tab[l] for l in true_labels]

In [16]:
def makePlot(idx, offset):
    cols = {
        "convnext": 0,
        # column one is for the actual image!
        "deit": 2,
        "swin": 3
    }
    rows = {
        "ots": 1,
        "ft": 0
    }
    
    gray = np.load(source_path + f"img{idx:03}_gray.npy")
    col = np.load(source_path + f"img{idx:03}.npy")
    plt.imshow(col, extent=[1+offset,2+offset,0.5+0.5*offset,1.5+0.5*offset])
    plt.text(1+0.1, 0.5+0.5*offset+0.1, true_labels[idx], bbox={'facecolor': 'white', 'alpha':0.7})

    for col in cols:
        for row in rows:
            map = np.load(source_path +  f"{col}_{row}_img{idx:03}_saliency.npy")
            
            # For deit still need to take class token attention
            if map.shape[0] == 197:
                map = map[0, 1:].reshape((14,14))

            pred = stringFromFile(source_path + f"{col}_{row}_img{idx:03}_prediction.txt")
            plotSalience(gray, map, pred, cols[col] * (1 + offset), rows[row] * (1 + offset))
    
    plt.xlim(left=0)
    x_ticks = []
    x_labels = []
    for col in cols:
        x_ticks += [cols[col] * (1 + offset) + 0.5]
        x_labels += [col]
    plt.xticks(x_ticks, x_labels)

    plt.ylim(top = 2 + offset)
    plt.yticks([0.5, 1.5 + offset], labels=["ft", "ots"])

    plt.gcf().set_size_inches(7,4)

    # plt.show()
    plt.savefig(target_dir + f"img{idx:03}_salience.png", format="png", dpi=600)
    plt.clf()

for idx in range(20):
    makePlot(idx, 0.05)

<Figure size 504x288 with 0 Axes>